In [54]:
# from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2Model
# import librosa
# import soundfile as sf
# import torch
# import pandas as pd

In [55]:
# import torch
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# print(device)

In [56]:
# processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
# model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-large-960h-lv60-self").to(device)
# model.config.output_hidden_states = True

In [57]:
# file_path = 'path_test.txt'
# with open(file_path, 'r') as file:
#     audio_paths = file.readlines()

In [58]:
# from tqdm import tqdm

# for audio_path in tqdm(audio_paths):
#     audio_path = audio_path.strip()

#     audio_input, sample_rate = sf.read(audio_path)

#     input_values = processor(audio_input, sampling_rate=sample_rate, return_tensors="pt").input_values.to(device)

#     with torch.no_grad():
#         outputs = model(input_values)
        
# #     with torch.no_grad():
# #     outputs = model(**inputs)

#     # Get the last hidden state features
#     last_hidden_state_features = outputs.last_hidden_state.squeeze()

#     # Get the features from all layers
#     all_hidden_states = outputs.hidden_states

In [59]:
# import pandas as pd

# # Flatten the last hidden state features
# last_hidden_state_flat = last_hidden_state_features.view(-1, last_hidden_state_features.size(-1)).numpy()

# # Convert to DataFrame
# df_last_hidden_state = pd.DataFrame(last_hidden_state_flat)

# # Save the last hidden state features to a CSV file
# df_last_hidden_state.to_csv("last_hidden_state_features.csv", index=False)

# # Prepare DataFrames for all layers
# dfs_all_layers = []
# for i, layer_features in enumerate(all_hidden_states):
#     layer_flat = layer_features.view(-1, layer_features.size(-1)).numpy()
#     df_layer = pd.DataFrame(layer_flat)
#     dfs_all_layers.append(df_layer)

#     # Save each layer's features to a CSV file
#     df_layer.to_csv(f"layer_{i}_features.csv", index=False)

In [62]:
from transformers import Wav2Vec2Processor, Wav2Vec2Model
import soundfile as sf
import torch
from tqdm import tqdm
import kaldiio

# Load the processor and model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model.config.output_hidden_states = True

# Specify the target layer number (0 for embeddings, 1 for first hidden layer, etc.)
target_layer = 1  # Adjust this according to which layer you want to extract features from

# Use CUDA if available
device = torch.device("cpu")
model.to(device)

# Read the list of audio file paths with speaker IDs
file_path = 'train/wav.scp'
with open(file_path, 'r') as file:
    audio_files = file.readlines()

# Lists to store speaker IDs and features for the target layer
speaker_ids = []
layer_features = []

# Process each audio file
for audio_file in tqdm(audio_files):
    audio_file = audio_file.strip()
   
    # Extract speaker ID from the file path
    speaker_id = audio_file.split('/')[-1].split('-')[0]  # adjust based on your file path format
    speaker_ids.append(speaker_id)
   
    # Load the audio file
    audio_input, sample_rate = sf.read(audio_file)
   
    # Process the audio input
    input_values = processor(audio_input, sampling_rate=sample_rate, return_tensors="pt").input_values.to(device)
   
    # Pass through the model
    with torch.no_grad():
        outputs = model(input_values)
   
    # Get the features from the target layer
    all_hidden_states = outputs.hidden_states
    layer_feature = all_hidden_states[target_layer].squeeze(0).cpu().numpy()  # Remove batch dimension and move to CPU
    layer_features.append(layer_feature)
   
    # Release GPU memory
    del input_values, outputs
    torch.cuda.empty_cache()

# Save the features in Kaldi format
output_ark = f'layer_train_features/layer_{target_layer}_features_train.ark'
output_scp = f'layer_train_features/layer_{target_layer}_features_train.scp'

with kaldiio.WriteHelper(f'ark,scp:{output_ark},{output_scp}') as writer:
    for j, feature in enumerate(layer_features):
        writer[speaker_ids[j]] = feature

print(f"Features from layer {target_layer} saved in Kaldi format.")


In [ ]:
from transformers import Wav2Vec2Processor, Wav2Vec2Model
import soundfile as sf
import torch
from tqdm import tqdm
import kaldiio

# Load the processor and model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model.config.output_hidden_states = True

# Specify the target layer number (0 for embeddings, 1 for first hidden layer, etc.)
target_layer = 2  # Adjust this according to which layer you want to extract features from

# Use CUDA if available
device = torch.device("cpu")
model.to(device)

# Read the list of audio file paths with speaker IDs
file_path = 'train/wav.scp'
with open(file_path, 'r') as file:
    audio_files = file.readlines()

# Lists to store speaker IDs and features for the target layer
speaker_ids = []
layer_features = []

# Process each audio file
for audio_file in tqdm(audio_files):
    audio_file = audio_file.strip()
   
    # Extract speaker ID from the file path
    speaker_id = audio_file.split('/')[-1].split('-')[0]  # adjust based on your file path format
    speaker_ids.append(speaker_id)
   
    # Load the audio file
    audio_input, sample_rate = sf.read(audio_file)
   
    # Process the audio input
    input_values = processor(audio_input, sampling_rate=sample_rate, return_tensors="pt").input_values.to(device)
   
    # Pass through the model
    with torch.no_grad():
        outputs = model(input_values)
   
    # Get the features from the target layer
    all_hidden_states = outputs.hidden_states
    layer_feature = all_hidden_states[target_layer].squeeze(0).cpu().numpy()  # Remove batch dimension and move to CPU
    layer_features.append(layer_feature)
   
    # Release GPU memory
    del input_values, outputs
    torch.cuda.empty_cache()

# Save the features in Kaldi format
output_ark = f'layer_train_features/layer_{target_layer}_features_train.ark'
output_scp = f'layer_train_features/layer_{target_layer}_features_train.scp'

with kaldiio.WriteHelper(f'ark,scp:{output_ark},{output_scp}') as writer:
    for j, feature in enumerate(layer_features):
        writer[speaker_ids[j]] = feature

print(f"Features from layer {target_layer} saved in Kaldi format.")


Some weights of the model checkpoint at facebook/wav2vec2-large-960h-lv60-self were not used when initializing Wav2Vec2Model: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 62/62 [00:19<00:00,  3.13it/s]

Features from layer 2 saved in Kaldi format.


In [ ]:
from transformers import Wav2Vec2Processor, Wav2Vec2Model
import soundfile as sf
import torch
from tqdm import tqdm
import kaldiio

# Load the processor and model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model.config.output_hidden_states = True

# Specify the target layer number (0 for embeddings, 1 for first hidden layer, etc.)
target_layer = 3  # Adjust this according to which layer you want to extract features from

# Use CUDA if available
device = torch.device("cpu")
model.to(device)

# Read the list of audio file paths with speaker IDs
file_path = 'train/wav.scp'
with open(file_path, 'r') as file:
    audio_files = file.readlines()

# Lists to store speaker IDs and features for the target layer
speaker_ids = []
layer_features = []

# Process each audio file
for audio_file in tqdm(audio_files):
    audio_file = audio_file.strip()
   
    # Extract speaker ID from the file path
    speaker_id = audio_file.split('/')[-1].split('-')[0]  # adjust based on your file path format
    speaker_ids.append(speaker_id)
   
    # Load the audio file
    audio_input, sample_rate = sf.read(audio_file)
   
    # Process the audio input
    input_values = processor(audio_input, sampling_rate=sample_rate, return_tensors="pt").input_values.to(device)
   
    # Pass through the model
    with torch.no_grad():
        outputs = model(input_values)
   
    # Get the features from the target layer
    all_hidden_states = outputs.hidden_states
    layer_feature = all_hidden_states[target_layer].squeeze(0).cpu().numpy()  # Remove batch dimension and move to CPU
    layer_features.append(layer_feature)
   
    # Release GPU memory
    del input_values, outputs
    torch.cuda.empty_cache()

# Save the features in Kaldi format
output_ark = f'layer_train_features/layer_{target_layer}_features_train.ark'
output_scp = f'layer_train_features/layer_{target_layer}_features_train.scp'

with kaldiio.WriteHelper(f'ark,scp:{output_ark},{output_scp}') as writer:
    for j, feature in enumerate(layer_features):
        writer[speaker_ids[j]] = feature

print(f"Features from layer {target_layer} saved in Kaldi format.")


Some weights of the model checkpoint at facebook/wav2vec2-large-960h-lv60-self were not used when initializing Wav2Vec2Model: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 62/62 [00:19<00:00,  3.16it/s]

Features from layer 3 saved in Kaldi format.


In [ ]:
from transformers import Wav2Vec2Processor, Wav2Vec2Model
import soundfile as sf
import torch
from tqdm import tqdm
import kaldiio

# Load the processor and model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model.config.output_hidden_states = True

# Specify the target layer number (0 for embeddings, 1 for first hidden layer, etc.)
target_layer = 4  # Adjust this according to which layer you want to extract features from

# Use CUDA if available
device = torch.device("cpu")
model.to(device)

# Read the list of audio file paths with speaker IDs
file_path = 'train/wav.scp'
with open(file_path, 'r') as file:
    audio_files = file.readlines()

# Lists to store speaker IDs and features for the target layer
speaker_ids = []
layer_features = []

# Process each audio file
for audio_file in tqdm(audio_files):
    audio_file = audio_file.strip()
   
    # Extract speaker ID from the file path
    speaker_id = audio_file.split('/')[-1].split('-')[0]  # adjust based on your file path format
    speaker_ids.append(speaker_id)
   
    # Load the audio file
    audio_input, sample_rate = sf.read(audio_file)
   
    # Process the audio input
    input_values = processor(audio_input, sampling_rate=sample_rate, return_tensors="pt").input_values.to(device)
   
    # Pass through the model
    with torch.no_grad():
        outputs = model(input_values)
   
    # Get the features from the target layer
    all_hidden_states = outputs.hidden_states
    layer_feature = all_hidden_states[target_layer].squeeze(0).cpu().numpy()  # Remove batch dimension and move to CPU
    layer_features.append(layer_feature)
   
    # Release GPU memory
    del input_values, outputs
    torch.cuda.empty_cache()

# Save the features in Kaldi format
output_ark = f'layer_train_features/layer_{target_layer}_features_train.ark'
output_scp = f'layer_train_features/layer_{target_layer}_features_train.scp'

with kaldiio.WriteHelper(f'ark,scp:{output_ark},{output_scp}') as writer:
    for j, feature in enumerate(layer_features):
        writer[speaker_ids[j]] = feature

print(f"Features from layer {target_layer} saved in Kaldi format.")


Some weights of the model checkpoint at facebook/wav2vec2-large-960h-lv60-self were not used when initializing Wav2Vec2Model: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 62/62 [00:19<00:00,  3.17it/s]

Features from layer 4 saved in Kaldi format.


In [ ]:
from transformers import Wav2Vec2Processor, Wav2Vec2Model
import soundfile as sf
import torch
from tqdm import tqdm
import kaldiio

# Load the processor and model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model.config.output_hidden_states = True

# Specify the target layer number (0 for embeddings, 1 for first hidden layer, etc.)
target_layer = 5  # Adjust this according to which layer you want to extract features from

# Use CUDA if available
device = torch.device("cpu")
model.to(device)

# Read the list of audio file paths with speaker IDs
file_path = 'train/wav.scp'
with open(file_path, 'r') as file:
    audio_files = file.readlines()

# Lists to store speaker IDs and features for the target layer
speaker_ids = []
layer_features = []

# Process each audio file
for audio_file in tqdm(audio_files):
    audio_file = audio_file.strip()
   
    # Extract speaker ID from the file path
    speaker_id = audio_file.split('/')[-1].split('-')[0]  # adjust based on your file path format
    speaker_ids.append(speaker_id)
   
    # Load the audio file
    audio_input, sample_rate = sf.read(audio_file)
   
    # Process the audio input
    input_values = processor(audio_input, sampling_rate=sample_rate, return_tensors="pt").input_values.to(device)
   
    # Pass through the model
    with torch.no_grad():
        outputs = model(input_values)
   
    # Get the features from the target layer
    all_hidden_states = outputs.hidden_states
    layer_feature = all_hidden_states[target_layer].squeeze(0).cpu().numpy()  # Remove batch dimension and move to CPU
    layer_features.append(layer_feature)
   
    # Release GPU memory
    del input_values, outputs
    torch.cuda.empty_cache()

# Save the features in Kaldi format
output_ark = f'layer_train_features/layer_{target_layer}_features_train.ark'
output_scp = f'layer_train_features/layer_{target_layer}_features_train.scp'

with kaldiio.WriteHelper(f'ark,scp:{output_ark},{output_scp}') as writer:
    for j, feature in enumerate(layer_features):
        writer[speaker_ids[j]] = feature

print(f"Features from layer {target_layer} saved in Kaldi format.")


Some weights of the model checkpoint at facebook/wav2vec2-large-960h-lv60-self were not used when initializing Wav2Vec2Model: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 62/62 [00:20<00:00,  3.04it/s]

Features from layer 5 saved in Kaldi format.


In [ ]:
from transformers import Wav2Vec2Processor, Wav2Vec2Model
import soundfile as sf
import torch
from tqdm import tqdm
import kaldiio

# Load the processor and model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model.config.output_hidden_states = True

# Specify the target layer number (0 for embeddings, 1 for first hidden layer, etc.)
target_layer = 6  # Adjust this according to which layer you want to extract features from

# Use CUDA if available
device = torch.device("cpu")
model.to(device)

# Read the list of audio file paths with speaker IDs
file_path = 'train/wav.scp'
with open(file_path, 'r') as file:
    audio_files = file.readlines()

# Lists to store speaker IDs and features for the target layer
speaker_ids = []
layer_features = []

# Process each audio file
for audio_file in tqdm(audio_files):
    audio_file = audio_file.strip()
   
    # Extract speaker ID from the file path
    speaker_id = audio_file.split('/')[-1].split('-')[0]  # adjust based on your file path format
    speaker_ids.append(speaker_id)
   
    # Load the audio file
    audio_input, sample_rate = sf.read(audio_file)
   
    # Process the audio input
    input_values = processor(audio_input, sampling_rate=sample_rate, return_tensors="pt").input_values.to(device)
   
    # Pass through the model
    with torch.no_grad():
        outputs = model(input_values)
   
    # Get the features from the target layer
    all_hidden_states = outputs.hidden_states
    layer_feature = all_hidden_states[target_layer].squeeze(0).cpu().numpy()  # Remove batch dimension and move to CPU
    layer_features.append(layer_feature)
   
    # Release GPU memory
    del input_values, outputs
    torch.cuda.empty_cache()

# Save the features in Kaldi format
output_ark = f'layer_train_features/layer_{target_layer}_features_train.ark'
output_scp = f'layer_train_features/layer_{target_layer}_features_train.scp'

with kaldiio.WriteHelper(f'ark,scp:{output_ark},{output_scp}') as writer:
    for j, feature in enumerate(layer_features):
        writer[speaker_ids[j]] = feature

print(f"Features from layer {target_layer} saved in Kaldi format.")


Some weights of the model checkpoint at facebook/wav2vec2-large-960h-lv60-self were not used when initializing Wav2Vec2Model: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 62/62 [00:19<00:00,  3.17it/s]

Features from layer 6 saved in Kaldi format.


In [ ]:
from transformers import Wav2Vec2Processor, Wav2Vec2Model
import soundfile as sf
import torch
from tqdm import tqdm
import kaldiio

# Load the processor and model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model.config.output_hidden_states = True

# Specify the target layer number (0 for embeddings, 1 for first hidden layer, etc.)
target_layer = 7  # Adjust this according to which layer you want to extract features from

# Use CUDA if available
device = torch.device("cpu")
model.to(device)

# Read the list of audio file paths with speaker IDs
file_path = 'train/wav.scp'
with open(file_path, 'r') as file:
    audio_files = file.readlines()

# Lists to store speaker IDs and features for the target layer
speaker_ids = []
layer_features = []

# Process each audio file
for audio_file in tqdm(audio_files):
    audio_file = audio_file.strip()
   
    # Extract speaker ID from the file path
    speaker_id = audio_file.split('/')[-1].split('-')[0]  # adjust based on your file path format
    speaker_ids.append(speaker_id)
   
    # Load the audio file
    audio_input, sample_rate = sf.read(audio_file)
   
    # Process the audio input
    input_values = processor(audio_input, sampling_rate=sample_rate, return_tensors="pt").input_values.to(device)
   
    # Pass through the model
    with torch.no_grad():
        outputs = model(input_values)
   
    # Get the features from the target layer
    all_hidden_states = outputs.hidden_states
    layer_feature = all_hidden_states[target_layer].squeeze(0).cpu().numpy()  # Remove batch dimension and move to CPU
    layer_features.append(layer_feature)
   
    # Release GPU memory
    del input_values, outputs
    torch.cuda.empty_cache()

# Save the features in Kaldi format
output_ark = f'layer_train_features/layer_{target_layer}_features_train.ark'
output_scp = f'layer_train_features/layer_{target_layer}_features_train.scp'

with kaldiio.WriteHelper(f'ark,scp:{output_ark},{output_scp}') as writer:
    for j, feature in enumerate(layer_features):
        writer[speaker_ids[j]] = feature

print(f"Features from layer {target_layer} saved in Kaldi format.")


Some weights of the model checkpoint at facebook/wav2vec2-large-960h-lv60-self were not used when initializing Wav2Vec2Model: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 62/62 [00:19<00:00,  3.12it/s]

Features from layer 7 saved in Kaldi format.


In [ ]:
from transformers import Wav2Vec2Processor, Wav2Vec2Model
import soundfile as sf
import torch
from tqdm import tqdm
import kaldiio

# Load the processor and model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model.config.output_hidden_states = True

# Specify the target layer number (0 for embeddings, 1 for first hidden layer, etc.)
target_layer = 8  # Adjust this according to which layer you want to extract features from

# Use CUDA if available
device = torch.device("cpu")
model.to(device)

# Read the list of audio file paths with speaker IDs
file_path = 'train/wav.scp'
with open(file_path, 'r') as file:
    audio_files = file.readlines()

# Lists to store speaker IDs and features for the target layer
speaker_ids = []
layer_features = []

# Process each audio file
for audio_file in tqdm(audio_files):
    audio_file = audio_file.strip()
   
    # Extract speaker ID from the file path
    speaker_id = audio_file.split('/')[-1].split('-')[0]  # adjust based on your file path format
    speaker_ids.append(speaker_id)
   
    # Load the audio file
    audio_input, sample_rate = sf.read(audio_file)
   
    # Process the audio input
    input_values = processor(audio_input, sampling_rate=sample_rate, return_tensors="pt").input_values.to(device)
   
    # Pass through the model
    with torch.no_grad():
        outputs = model(input_values)
   
    # Get the features from the target layer
    all_hidden_states = outputs.hidden_states
    layer_feature = all_hidden_states[target_layer].squeeze(0).cpu().numpy()  # Remove batch dimension and move to CPU
    layer_features.append(layer_feature)
   
    # Release GPU memory
    del input_values, outputs
    torch.cuda.empty_cache()

# Save the features in Kaldi format
output_ark = f'layer_train_features/layer_{target_layer}_features_train.ark'
output_scp = f'layer_train_features/layer_{target_layer}_features_train.scp'

with kaldiio.WriteHelper(f'ark,scp:{output_ark},{output_scp}') as writer:
    for j, feature in enumerate(layer_features):
        writer[speaker_ids[j]] = feature

print(f"Features from layer {target_layer} saved in Kaldi format.")


Some weights of the model checkpoint at facebook/wav2vec2-large-960h-lv60-self were not used when initializing Wav2Vec2Model: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 62/62 [00:20<00:00,  3.00it/s]

Features from layer 8 saved in Kaldi format.


In [ ]:
from transformers import Wav2Vec2Processor, Wav2Vec2Model
import soundfile as sf
import torch
from tqdm import tqdm
import kaldiio

# Load the processor and model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model.config.output_hidden_states = True

# Specify the target layer number (0 for embeddings, 1 for first hidden layer, etc.)
target_layer = 9  # Adjust this according to which layer you want to extract features from

# Use CUDA if available
device = torch.device("cpu")
model.to(device)

# Read the list of audio file paths with speaker IDs
file_path = 'train/wav.scp'
with open(file_path, 'r') as file:
    audio_files = file.readlines()

# Lists to store speaker IDs and features for the target layer
speaker_ids = []
layer_features = []

# Process each audio file
for audio_file in tqdm(audio_files):
    audio_file = audio_file.strip()
   
    # Extract speaker ID from the file path
    speaker_id = audio_file.split('/')[-1].split('-')[0]  # adjust based on your file path format
    speaker_ids.append(speaker_id)
   
    # Load the audio file
    audio_input, sample_rate = sf.read(audio_file)
   
    # Process the audio input
    input_values = processor(audio_input, sampling_rate=sample_rate, return_tensors="pt").input_values.to(device)
   
    # Pass through the model
    with torch.no_grad():
        outputs = model(input_values)
   
    # Get the features from the target layer
    all_hidden_states = outputs.hidden_states
    layer_feature = all_hidden_states[target_layer].squeeze(0).cpu().numpy()  # Remove batch dimension and move to CPU
    layer_features.append(layer_feature)
   
    # Release GPU memory
    del input_values, outputs
    torch.cuda.empty_cache()

# Save the features in Kaldi format
output_ark = f'layer_train_features/layer_{target_layer}_features_train.ark'
output_scp = f'layer_train_features/layer_{target_layer}_features_train.scp'

with kaldiio.WriteHelper(f'ark,scp:{output_ark},{output_scp}') as writer:
    for j, feature in enumerate(layer_features):
        writer[speaker_ids[j]] = feature

print(f"Features from layer {target_layer} saved in Kaldi format.")


Some weights of the model checkpoint at facebook/wav2vec2-large-960h-lv60-self were not used when initializing Wav2Vec2Model: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 62/62 [00:20<00:00,  3.00it/s]

Features from layer 9 saved in Kaldi format.


In [ ]:
from transformers import Wav2Vec2Processor, Wav2Vec2Model
import soundfile as sf
import torch
from tqdm import tqdm
import kaldiio

# Load the processor and model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model.config.output_hidden_states = True

# Specify the target layer number (0 for embeddings, 1 for first hidden layer, etc.)
target_layer = 10  # Adjust this according to which layer you want to extract features from

# Use CUDA if available
device = torch.device("cpu")
model.to(device)

# Read the list of audio file paths with speaker IDs
file_path = 'train/wav.scp'
with open(file_path, 'r') as file:
    audio_files = file.readlines()

# Lists to store speaker IDs and features for the target layer
speaker_ids = []
layer_features = []

# Process each audio file
for audio_file in tqdm(audio_files):
    audio_file = audio_file.strip()
   
    # Extract speaker ID from the file path
    speaker_id = audio_file.split('/')[-1].split('-')[0]  # adjust based on your file path format
    speaker_ids.append(speaker_id)
   
    # Load the audio file
    audio_input, sample_rate = sf.read(audio_file)
   
    # Process the audio input
    input_values = processor(audio_input, sampling_rate=sample_rate, return_tensors="pt").input_values.to(device)
   
    # Pass through the model
    with torch.no_grad():
        outputs = model(input_values)
   
    # Get the features from the target layer
    all_hidden_states = outputs.hidden_states
    layer_feature = all_hidden_states[target_layer].squeeze(0).cpu().numpy()  # Remove batch dimension and move to CPU
    layer_features.append(layer_feature)
   
    # Release GPU memory
    del input_values, outputs
    torch.cuda.empty_cache()

# Save the features in Kaldi format
output_ark = f'layer_train_features/layer_{target_layer}_features_train.ark'
output_scp = f'layer_train_features/layer_{target_layer}_features_train.scp'

with kaldiio.WriteHelper(f'ark,scp:{output_ark},{output_scp}') as writer:
    for j, feature in enumerate(layer_features):
        writer[speaker_ids[j]] = feature

print(f"Features from layer {target_layer} saved in Kaldi format.")


Some weights of the model checkpoint at facebook/wav2vec2-large-960h-lv60-self were not used when initializing Wav2Vec2Model: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 62/62 [00:20<00:00,  3.06it/s]

Features from layer 10 saved in Kaldi format.


In [ ]:
from transformers import Wav2Vec2Processor, Wav2Vec2Model
import soundfile as sf
import torch
from tqdm import tqdm
import kaldiio

# Load the processor and model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model.config.output_hidden_states = True

# Specify the target layer number (0 for embeddings, 1 for first hidden layer, etc.)
target_layer = 11  # Adjust this according to which layer you want to extract features from

# Use CUDA if available
device = torch.device("cpu")
model.to(device)

# Read the list of audio file paths with speaker IDs
file_path = 'train/wav.scp'
with open(file_path, 'r') as file:
    audio_files = file.readlines()

# Lists to store speaker IDs and features for the target layer
speaker_ids = []
layer_features = []

# Process each audio file
for audio_file in tqdm(audio_files):
    audio_file = audio_file.strip()
   
    # Extract speaker ID from the file path
    speaker_id = audio_file.split('/')[-1].split('-')[0]  # adjust based on your file path format
    speaker_ids.append(speaker_id)
   
    # Load the audio file
    audio_input, sample_rate = sf.read(audio_file)
   
    # Process the audio input
    input_values = processor(audio_input, sampling_rate=sample_rate, return_tensors="pt").input_values.to(device)
   
    # Pass through the model
    with torch.no_grad():
        outputs = model(input_values)
   
    # Get the features from the target layer
    all_hidden_states = outputs.hidden_states
    layer_feature = all_hidden_states[target_layer].squeeze(0).cpu().numpy()  # Remove batch dimension and move to CPU
    layer_features.append(layer_feature)
   
    # Release GPU memory
    del input_values, outputs
    torch.cuda.empty_cache()

# Save the features in Kaldi format
output_ark = f'layer_train_features/layer_{target_layer}_features_train.ark'
output_scp = f'layer_train_features/layer_{target_layer}_features_train.scp'

with kaldiio.WriteHelper(f'ark,scp:{output_ark},{output_scp}') as writer:
    for j, feature in enumerate(layer_features):
        writer[speaker_ids[j]] = feature

print(f"Features from layer {target_layer} saved in Kaldi format.")


Some weights of the model checkpoint at facebook/wav2vec2-large-960h-lv60-self were not used when initializing Wav2Vec2Model: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 62/62 [00:19<00:00,  3.12it/s]

Features from layer 11 saved in Kaldi format.


In [ ]:
from transformers import Wav2Vec2Processor, Wav2Vec2Model
import soundfile as sf
import torch
from tqdm import tqdm
import kaldiio

# Load the processor and model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model.config.output_hidden_states = True

# Specify the target layer number (0 for embeddings, 1 for first hidden layer, etc.)
target_layer = 12  # Adjust this according to which layer you want to extract features from

# Use CUDA if available
device = torch.device("cpu")
model.to(device)

# Read the list of audio file paths with speaker IDs
file_path = 'train/wav.scp'
with open(file_path, 'r') as file:
    audio_files = file.readlines()

# Lists to store speaker IDs and features for the target layer
speaker_ids = []
layer_features = []

# Process each audio file
for audio_file in tqdm(audio_files):
    audio_file = audio_file.strip()
   
    # Extract speaker ID from the file path
    speaker_id = audio_file.split('/')[-1].split('-')[0]  # adjust based on your file path format
    speaker_ids.append(speaker_id)
   
    # Load the audio file
    audio_input, sample_rate = sf.read(audio_file)
   
    # Process the audio input
    input_values = processor(audio_input, sampling_rate=sample_rate, return_tensors="pt").input_values.to(device)
   
    # Pass through the model
    with torch.no_grad():
        outputs = model(input_values)
   
    # Get the features from the target layer
    all_hidden_states = outputs.hidden_states
    layer_feature = all_hidden_states[target_layer].squeeze(0).cpu().numpy()  # Remove batch dimension and move to CPU
    layer_features.append(layer_feature)
   
    # Release GPU memory
    del input_values, outputs
    torch.cuda.empty_cache()

# Save the features in Kaldi format
output_ark = f'layer_train_features/layer_{target_layer}_features_train.ark'
output_scp = f'layer_train_features/layer_{target_layer}_features_train.scp'

with kaldiio.WriteHelper(f'ark,scp:{output_ark},{output_scp}') as writer:
    for j, feature in enumerate(layer_features):
        writer[speaker_ids[j]] = feature

print(f"Features from layer {target_layer} saved in Kaldi format.")


Some weights of the model checkpoint at facebook/wav2vec2-large-960h-lv60-self were not used when initializing Wav2Vec2Model: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 62/62 [00:20<00:00,  3.01it/s]

Features from layer 12 saved in Kaldi format.


In [ ]:
from transformers import Wav2Vec2Processor, Wav2Vec2Model
import soundfile as sf
import torch
from tqdm import tqdm
import kaldiio

# Load the processor and model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model.config.output_hidden_states = True

# Specify the target layer number (0 for embeddings, 1 for first hidden layer, etc.)
target_layer = 13  # Adjust this according to which layer you want to extract features from

# Use CUDA if available
device = torch.device("cpu")
model.to(device)

# Read the list of audio file paths with speaker IDs
file_path = 'train/wav.scp'
with open(file_path, 'r') as file:
    audio_files = file.readlines()

# Lists to store speaker IDs and features for the target layer
speaker_ids = []
layer_features = []

# Process each audio file
for audio_file in tqdm(audio_files):
    audio_file = audio_file.strip()
   
    # Extract speaker ID from the file path
    speaker_id = audio_file.split('/')[-1].split('-')[0]  # adjust based on your file path format
    speaker_ids.append(speaker_id)
   
    # Load the audio file
    audio_input, sample_rate = sf.read(audio_file)
   
    # Process the audio input
    input_values = processor(audio_input, sampling_rate=sample_rate, return_tensors="pt").input_values.to(device)
   
    # Pass through the model
    with torch.no_grad():
        outputs = model(input_values)
   
    # Get the features from the target layer
    all_hidden_states = outputs.hidden_states
    layer_feature = all_hidden_states[target_layer].squeeze(0).cpu().numpy()  # Remove batch dimension and move to CPU
    layer_features.append(layer_feature)
   
    # Release GPU memory
    del input_values, outputs
    torch.cuda.empty_cache()

# Save the features in Kaldi format
output_ark = f'layer_train_features/layer_{target_layer}_features_train.ark'
output_scp = f'layer_train_features/layer_{target_layer}_features_train.scp'

with kaldiio.WriteHelper(f'ark,scp:{output_ark},{output_scp}') as writer:
    for j, feature in enumerate(layer_features):
        writer[speaker_ids[j]] = feature

print(f"Features from layer {target_layer} saved in Kaldi format.")


Some weights of the model checkpoint at facebook/wav2vec2-large-960h-lv60-self were not used when initializing Wav2Vec2Model: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 62/62 [00:20<00:00,  3.02it/s]

Features from layer 13 saved in Kaldi format.


In [ ]:
from transformers import Wav2Vec2Processor, Wav2Vec2Model
import soundfile as sf
import torch
from tqdm import tqdm
import kaldiio

# Load the processor and model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model.config.output_hidden_states = True

# Specify the target layer number (0 for embeddings, 1 for first hidden layer, etc.)
target_layer = 14  # Adjust this according to which layer you want to extract features from

# Use CUDA if available
device = torch.device("cpu")
model.to(device)

# Read the list of audio file paths with speaker IDs
file_path = 'train/wav.scp'
with open(file_path, 'r') as file:
    audio_files = file.readlines()

# Lists to store speaker IDs and features for the target layer
speaker_ids = []
layer_features = []

# Process each audio file
for audio_file in tqdm(audio_files):
    audio_file = audio_file.strip()
   
    # Extract speaker ID from the file path
    speaker_id = audio_file.split('/')[-1].split('-')[0]  # adjust based on your file path format
    speaker_ids.append(speaker_id)
   
    # Load the audio file
    audio_input, sample_rate = sf.read(audio_file)
   
    # Process the audio input
    input_values = processor(audio_input, sampling_rate=sample_rate, return_tensors="pt").input_values.to(device)
   
    # Pass through the model
    with torch.no_grad():
        outputs = model(input_values)
   
    # Get the features from the target layer
    all_hidden_states = outputs.hidden_states
    layer_feature = all_hidden_states[target_layer].squeeze(0).cpu().numpy()  # Remove batch dimension and move to CPU
    layer_features.append(layer_feature)
   
    # Release GPU memory
    del input_values, outputs
    torch.cuda.empty_cache()

# Save the features in Kaldi format
output_ark = f'layer_train_features/layer_{target_layer}_features_train.ark'
output_scp = f'layer_train_features/layer_{target_layer}_features_train.scp'

with kaldiio.WriteHelper(f'ark,scp:{output_ark},{output_scp}') as writer:
    for j, feature in enumerate(layer_features):
        writer[speaker_ids[j]] = feature

print(f"Features from layer {target_layer} saved in Kaldi format.")


Some weights of the model checkpoint at facebook/wav2vec2-large-960h-lv60-self were not used when initializing Wav2Vec2Model: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 62/62 [00:20<00:00,  3.08it/s]

Features from layer 14 saved in Kaldi format.


In [ ]:
from transformers import Wav2Vec2Processor, Wav2Vec2Model
import soundfile as sf
import torch
from tqdm import tqdm
import kaldiio

# Load the processor and model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model.config.output_hidden_states = True

# Specify the target layer number (0 for embeddings, 1 for first hidden layer, etc.)
target_layer = 15  # Adjust this according to which layer you want to extract features from

# Use CUDA if available
device = torch.device("cpu")
model.to(device)

# Read the list of audio file paths with speaker IDs
file_path = 'train/wav.scp'
with open(file_path, 'r') as file:
    audio_files = file.readlines()

# Lists to store speaker IDs and features for the target layer
speaker_ids = []
layer_features = []

# Process each audio file
for audio_file in tqdm(audio_files):
    audio_file = audio_file.strip()
   
    # Extract speaker ID from the file path
    speaker_id = audio_file.split('/')[-1].split('-')[0]  # adjust based on your file path format
    speaker_ids.append(speaker_id)
   
    # Load the audio file
    audio_input, sample_rate = sf.read(audio_file)
   
    # Process the audio input
    input_values = processor(audio_input, sampling_rate=sample_rate, return_tensors="pt").input_values.to(device)
   
    # Pass through the model
    with torch.no_grad():
        outputs = model(input_values)
   
    # Get the features from the target layer
    all_hidden_states = outputs.hidden_states
    layer_feature = all_hidden_states[target_layer].squeeze(0).cpu().numpy()  # Remove batch dimension and move to CPU
    layer_features.append(layer_feature)
   
    # Release GPU memory
    del input_values, outputs
    torch.cuda.empty_cache()

# Save the features in Kaldi format
output_ark = f'layer_train_features/layer_{target_layer}_features_train.ark'
output_scp = f'layer_train_features/layer_{target_layer}_features_train.scp'

with kaldiio.WriteHelper(f'ark,scp:{output_ark},{output_scp}') as writer:
    for j, feature in enumerate(layer_features):
        writer[speaker_ids[j]] = feature

print(f"Features from layer {target_layer} saved in Kaldi format.")


Some weights of the model checkpoint at facebook/wav2vec2-large-960h-lv60-self were not used when initializing Wav2Vec2Model: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 62/62 [00:20<00:00,  3.05it/s]

Features from layer 15 saved in Kaldi format.


In [ ]:
from transformers import Wav2Vec2Processor, Wav2Vec2Model
import soundfile as sf
import torch
from tqdm import tqdm
import kaldiio

# Load the processor and model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model.config.output_hidden_states = True

# Specify the target layer number (0 for embeddings, 1 for first hidden layer, etc.)
target_layer = 16  # Adjust this according to which layer you want to extract features from

# Use CUDA if available
device = torch.device("cpu")
model.to(device)

# Read the list of audio file paths with speaker IDs
file_path = 'train/wav.scp'
with open(file_path, 'r') as file:
    audio_files = file.readlines()

# Lists to store speaker IDs and features for the target layer
speaker_ids = []
layer_features = []

# Process each audio file
for audio_file in tqdm(audio_files):
    audio_file = audio_file.strip()
   
    # Extract speaker ID from the file path
    speaker_id = audio_file.split('/')[-1].split('-')[0]  # adjust based on your file path format
    speaker_ids.append(speaker_id)
   
    # Load the audio file
    audio_input, sample_rate = sf.read(audio_file)
   
    # Process the audio input
    input_values = processor(audio_input, sampling_rate=sample_rate, return_tensors="pt").input_values.to(device)
   
    # Pass through the model
    with torch.no_grad():
        outputs = model(input_values)
   
    # Get the features from the target layer
    all_hidden_states = outputs.hidden_states
    layer_feature = all_hidden_states[target_layer].squeeze(0).cpu().numpy()  # Remove batch dimension and move to CPU
    layer_features.append(layer_feature)
   
    # Release GPU memory
    del input_values, outputs
    torch.cuda.empty_cache()

# Save the features in Kaldi format
output_ark = f'layer_train_features/layer_{target_layer}_features_train.ark'
output_scp = f'layer_train_features/layer_{target_layer}_features_train.scp'

with kaldiio.WriteHelper(f'ark,scp:{output_ark},{output_scp}') as writer:
    for j, feature in enumerate(layer_features):
        writer[speaker_ids[j]] = feature

print(f"Features from layer {target_layer} saved in Kaldi format.")


Some weights of the model checkpoint at facebook/wav2vec2-large-960h-lv60-self were not used when initializing Wav2Vec2Model: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 62/62 [00:20<00:00,  3.03it/s]

Features from layer 16 saved in Kaldi format.


In [ ]:
from transformers import Wav2Vec2Processor, Wav2Vec2Model
import soundfile as sf
import torch
from tqdm import tqdm
import kaldiio

# Load the processor and model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model.config.output_hidden_states = True

# Specify the target layer number (0 for embeddings, 1 for first hidden layer, etc.)
target_layer = 17  # Adjust this according to which layer you want to extract features from

# Use CUDA if available
device = torch.device("cpu")
model.to(device)

# Read the list of audio file paths with speaker IDs
file_path = 'train/wav.scp'
with open(file_path, 'r') as file:
    audio_files = file.readlines()

# Lists to store speaker IDs and features for the target layer
speaker_ids = []
layer_features = []

# Process each audio file
for audio_file in tqdm(audio_files):
    audio_file = audio_file.strip()
   
    # Extract speaker ID from the file path
    speaker_id = audio_file.split('/')[-1].split('-')[0]  # adjust based on your file path format
    speaker_ids.append(speaker_id)
   
    # Load the audio file
    audio_input, sample_rate = sf.read(audio_file)
   
    # Process the audio input
    input_values = processor(audio_input, sampling_rate=sample_rate, return_tensors="pt").input_values.to(device)
   
    # Pass through the model
    with torch.no_grad():
        outputs = model(input_values)
   
    # Get the features from the target layer
    all_hidden_states = outputs.hidden_states
    layer_feature = all_hidden_states[target_layer].squeeze(0).cpu().numpy()  # Remove batch dimension and move to CPU
    layer_features.append(layer_feature)
   
    # Release GPU memory
    del input_values, outputs
    torch.cuda.empty_cache()

# Save the features in Kaldi format
output_ark = f'layer_train_features/layer_{target_layer}_features_train.ark'
output_scp = f'layer_train_features/layer_{target_layer}_features_train.scp'

with kaldiio.WriteHelper(f'ark,scp:{output_ark},{output_scp}') as writer:
    for j, feature in enumerate(layer_features):
        writer[speaker_ids[j]] = feature

print(f"Features from layer {target_layer} saved in Kaldi format.")


Some weights of the model checkpoint at facebook/wav2vec2-large-960h-lv60-self were not used when initializing Wav2Vec2Model: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 62/62 [00:20<00:00,  3.06it/s]

Features from layer 17 saved in Kaldi format.


In [ ]:
from transformers import Wav2Vec2Processor, Wav2Vec2Model
import soundfile as sf
import torch
from tqdm import tqdm
import kaldiio

# Load the processor and model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model.config.output_hidden_states = True

# Specify the target layer number (0 for embeddings, 1 for first hidden layer, etc.)
target_layer = 18  # Adjust this according to which layer you want to extract features from

# Use CUDA if available
device = torch.device("cpu")
model.to(device)

# Read the list of audio file paths with speaker IDs
file_path = 'train/wav.scp'
with open(file_path, 'r') as file:
    audio_files = file.readlines()

# Lists to store speaker IDs and features for the target layer
speaker_ids = []
layer_features = []

# Process each audio file
for audio_file in tqdm(audio_files):
    audio_file = audio_file.strip()
   
    # Extract speaker ID from the file path
    speaker_id = audio_file.split('/')[-1].split('-')[0]  # adjust based on your file path format
    speaker_ids.append(speaker_id)
   
    # Load the audio file
    audio_input, sample_rate = sf.read(audio_file)
   
    # Process the audio input
    input_values = processor(audio_input, sampling_rate=sample_rate, return_tensors="pt").input_values.to(device)
   
    # Pass through the model
    with torch.no_grad():
        outputs = model(input_values)
   
    # Get the features from the target layer
    all_hidden_states = outputs.hidden_states
    layer_feature = all_hidden_states[target_layer].squeeze(0).cpu().numpy()  # Remove batch dimension and move to CPU
    layer_features.append(layer_feature)
   
    # Release GPU memory
    del input_values, outputs
    torch.cuda.empty_cache()

# Save the features in Kaldi format
output_ark = f'layer_train_features/layer_{target_layer}_features_train.ark'
output_scp = f'layer_train_features/layer_{target_layer}_features_train.scp'

with kaldiio.WriteHelper(f'ark,scp:{output_ark},{output_scp}') as writer:
    for j, feature in enumerate(layer_features):
        writer[speaker_ids[j]] = feature

print(f"Features from layer {target_layer} saved in Kaldi format.")


Some weights of the model checkpoint at facebook/wav2vec2-large-960h-lv60-self were not used when initializing Wav2Vec2Model: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 62/62 [00:19<00:00,  3.10it/s]

Features from layer 18 saved in Kaldi format.


In [ ]:
from transformers import Wav2Vec2Processor, Wav2Vec2Model
import soundfile as sf
import torch
from tqdm import tqdm
import kaldiio

# Load the processor and model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model.config.output_hidden_states = True

# Specify the target layer number (0 for embeddings, 1 for first hidden layer, etc.)
target_layer = 19  # Adjust this according to which layer you want to extract features from

# Use CUDA if available
device = torch.device("cpu")
model.to(device)

# Read the list of audio file paths with speaker IDs
file_path = 'train/wav.scp'
with open(file_path, 'r') as file:
    audio_files = file.readlines()

# Lists to store speaker IDs and features for the target layer
speaker_ids = []
layer_features = []

# Process each audio file
for audio_file in tqdm(audio_files):
    audio_file = audio_file.strip()
   
    # Extract speaker ID from the file path
    speaker_id = audio_file.split('/')[-1].split('-')[0]  # adjust based on your file path format
    speaker_ids.append(speaker_id)
   
    # Load the audio file
    audio_input, sample_rate = sf.read(audio_file)
   
    # Process the audio input
    input_values = processor(audio_input, sampling_rate=sample_rate, return_tensors="pt").input_values.to(device)
   
    # Pass through the model
    with torch.no_grad():
        outputs = model(input_values)
   
    # Get the features from the target layer
    all_hidden_states = outputs.hidden_states
    layer_feature = all_hidden_states[target_layer].squeeze(0).cpu().numpy()  # Remove batch dimension and move to CPU
    layer_features.append(layer_feature)
   
    # Release GPU memory
    del input_values, outputs
    torch.cuda.empty_cache()

# Save the features in Kaldi format
output_ark = f'layer_train_features/layer_{target_layer}_features_train.ark'
output_scp = f'layer_train_features/layer_{target_layer}_features_train.scp'

with kaldiio.WriteHelper(f'ark,scp:{output_ark},{output_scp}') as writer:
    for j, feature in enumerate(layer_features):
        writer[speaker_ids[j]] = feature

print(f"Features from layer {target_layer} saved in Kaldi format.")


Some weights of the model checkpoint at facebook/wav2vec2-large-960h-lv60-self were not used when initializing Wav2Vec2Model: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 62/62 [00:20<00:00,  2.96it/s]

Features from layer 19 saved in Kaldi format.


In [ ]:
from transformers import Wav2Vec2Processor, Wav2Vec2Model
import soundfile as sf
import torch
from tqdm import tqdm
import kaldiio

# Load the processor and model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model.config.output_hidden_states = True

# Specify the target layer number (0 for embeddings, 1 for first hidden layer, etc.)
target_layer = 20  # Adjust this according to which layer you want to extract features from

# Use CUDA if available
device = torch.device("cpu")
model.to(device)

# Read the list of audio file paths with speaker IDs
file_path = 'train/wav.scp'
with open(file_path, 'r') as file:
    audio_files = file.readlines()

# Lists to store speaker IDs and features for the target layer
speaker_ids = []
layer_features = []

# Process each audio file
for audio_file in tqdm(audio_files):
    audio_file = audio_file.strip()
   
    # Extract speaker ID from the file path
    speaker_id = audio_file.split('/')[-1].split('-')[0]  # adjust based on your file path format
    speaker_ids.append(speaker_id)
   
    # Load the audio file
    audio_input, sample_rate = sf.read(audio_file)
   
    # Process the audio input
    input_values = processor(audio_input, sampling_rate=sample_rate, return_tensors="pt").input_values.to(device)
   
    # Pass through the model
    with torch.no_grad():
        outputs = model(input_values)
   
    # Get the features from the target layer
    all_hidden_states = outputs.hidden_states
    layer_feature = all_hidden_states[target_layer].squeeze(0).cpu().numpy()  # Remove batch dimension and move to CPU
    layer_features.append(layer_feature)
   
    # Release GPU memory
    del input_values, outputs
    torch.cuda.empty_cache()

# Save the features in Kaldi format
output_ark = f'layer_train_features/layer_{target_layer}_features_train.ark'
output_scp = f'layer_train_features/layer_{target_layer}_features_train.scp'

with kaldiio.WriteHelper(f'ark,scp:{output_ark},{output_scp}') as writer:
    for j, feature in enumerate(layer_features):
        writer[speaker_ids[j]] = feature

print(f"Features from layer {target_layer} saved in Kaldi format.")


Some weights of the model checkpoint at facebook/wav2vec2-large-960h-lv60-self were not used when initializing Wav2Vec2Model: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 62/62 [00:19<00:00,  3.14it/s]

Features from layer 20 saved in Kaldi format.


In [ ]:
from transformers import Wav2Vec2Processor, Wav2Vec2Model
import soundfile as sf
import torch
from tqdm import tqdm
import kaldiio

# Load the processor and model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model.config.output_hidden_states = True

# Specify the target layer number (0 for embeddings, 1 for first hidden layer, etc.)
target_layer = 21  # Adjust this according to which layer you want to extract features from

# Use CUDA if available
device = torch.device("cpu")
model.to(device)

# Read the list of audio file paths with speaker IDs
file_path = 'train/wav.scp'
with open(file_path, 'r') as file:
    audio_files = file.readlines()

# Lists to store speaker IDs and features for the target layer
speaker_ids = []
layer_features = []

# Process each audio file
for audio_file in tqdm(audio_files):
    audio_file = audio_file.strip()
   
    # Extract speaker ID from the file path
    speaker_id = audio_file.split('/')[-1].split('-')[0]  # adjust based on your file path format
    speaker_ids.append(speaker_id)
   
    # Load the audio file
    audio_input, sample_rate = sf.read(audio_file)
   
    # Process the audio input
    input_values = processor(audio_input, sampling_rate=sample_rate, return_tensors="pt").input_values.to(device)
   
    # Pass through the model
    with torch.no_grad():
        outputs = model(input_values)
   
    # Get the features from the target layer
    all_hidden_states = outputs.hidden_states
    layer_feature = all_hidden_states[target_layer].squeeze(0).cpu().numpy()  # Remove batch dimension and move to CPU
    layer_features.append(layer_feature)
   
    # Release GPU memory
    del input_values, outputs
    torch.cuda.empty_cache()

# Save the features in Kaldi format
output_ark = f'layer_train_features/layer_{target_layer}_features_train.ark'
output_scp = f'layer_train_features/layer_{target_layer}_features_train.scp'

with kaldiio.WriteHelper(f'ark,scp:{output_ark},{output_scp}') as writer:
    for j, feature in enumerate(layer_features):
        writer[speaker_ids[j]] = feature

print(f"Features from layer {target_layer} saved in Kaldi format.")


Some weights of the model checkpoint at facebook/wav2vec2-large-960h-lv60-self were not used when initializing Wav2Vec2Model: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 62/62 [00:20<00:00,  3.03it/s]

Features from layer 21 saved in Kaldi format.


In [ ]:
from transformers import Wav2Vec2Processor, Wav2Vec2Model
import soundfile as sf
import torch
from tqdm import tqdm
import kaldiio

# Load the processor and model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model.config.output_hidden_states = True

# Specify the target layer number (0 for embeddings, 1 for first hidden layer, etc.)
target_layer = 22  # Adjust this according to which layer you want to extract features from

# Use CUDA if available
device = torch.device("cpu")
model.to(device)

# Read the list of audio file paths with speaker IDs
file_path = 'train/wav.scp'
with open(file_path, 'r') as file:
    audio_files = file.readlines()

# Lists to store speaker IDs and features for the target layer
speaker_ids = []
layer_features = []

# Process each audio file
for audio_file in tqdm(audio_files):
    audio_file = audio_file.strip()
   
    # Extract speaker ID from the file path
    speaker_id = audio_file.split('/')[-1].split('-')[0]  # adjust based on your file path format
    speaker_ids.append(speaker_id)
   
    # Load the audio file
    audio_input, sample_rate = sf.read(audio_file)
   
    # Process the audio input
    input_values = processor(audio_input, sampling_rate=sample_rate, return_tensors="pt").input_values.to(device)
   
    # Pass through the model
    with torch.no_grad():
        outputs = model(input_values)
   
    # Get the features from the target layer
    all_hidden_states = outputs.hidden_states
    layer_feature = all_hidden_states[target_layer].squeeze(0).cpu().numpy()  # Remove batch dimension and move to CPU
    layer_features.append(layer_feature)
   
    # Release GPU memory
    del input_values, outputs
    torch.cuda.empty_cache()

# Save the features in Kaldi format
output_ark = f'layer_train_features/layer_{target_layer}_features_train.ark'
output_scp = f'layer_train_features/layer_{target_layer}_features_train.scp'

with kaldiio.WriteHelper(f'ark,scp:{output_ark},{output_scp}') as writer:
    for j, feature in enumerate(layer_features):
        writer[speaker_ids[j]] = feature

print(f"Features from layer {target_layer} saved in Kaldi format.")


Some weights of the model checkpoint at facebook/wav2vec2-large-960h-lv60-self were not used when initializing Wav2Vec2Model: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 62/62 [00:20<00:00,  3.08it/s]

Features from layer 22 saved in Kaldi format.


In [ ]:
from transformers import Wav2Vec2Processor, Wav2Vec2Model
import soundfile as sf
import torch
from tqdm import tqdm
import kaldiio

# Load the processor and model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model.config.output_hidden_states = True

# Specify the target layer number (0 for embeddings, 1 for first hidden layer, etc.)
target_layer = 23  # Adjust this according to which layer you want to extract features from

# Use CUDA if available
device = torch.device("cpu")
model.to(device)

# Read the list of audio file paths with speaker IDs
file_path = 'train/wav.scp'
with open(file_path, 'r') as file:
    audio_files = file.readlines()

# Lists to store speaker IDs and features for the target layer
speaker_ids = []
layer_features = []

# Process each audio file
for audio_file in tqdm(audio_files):
    audio_file = audio_file.strip()
   
    # Extract speaker ID from the file path
    speaker_id = audio_file.split('/')[-1].split('-')[0]  # adjust based on your file path format
    speaker_ids.append(speaker_id)
   
    # Load the audio file
    audio_input, sample_rate = sf.read(audio_file)
   
    # Process the audio input
    input_values = processor(audio_input, sampling_rate=sample_rate, return_tensors="pt").input_values.to(device)
   
    # Pass through the model
    with torch.no_grad():
        outputs = model(input_values)
   
    # Get the features from the target layer
    all_hidden_states = outputs.hidden_states
    layer_feature = all_hidden_states[target_layer].squeeze(0).cpu().numpy()  # Remove batch dimension and move to CPU
    layer_features.append(layer_feature)
   
    # Release GPU memory
    del input_values, outputs
    torch.cuda.empty_cache()

# Save the features in Kaldi format
output_ark = f'layer_train_features/layer_{target_layer}_features_train.ark'
output_scp = f'layer_train_features/layer_{target_layer}_features_train.scp'

with kaldiio.WriteHelper(f'ark,scp:{output_ark},{output_scp}') as writer:
    for j, feature in enumerate(layer_features):
        writer[speaker_ids[j]] = feature

print(f"Features from layer {target_layer} saved in Kaldi format.")


Some weights of the model checkpoint at facebook/wav2vec2-large-960h-lv60-self were not used when initializing Wav2Vec2Model: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 62/62 [00:21<00:00,  2.86it/s]

Features from layer 23 saved in Kaldi format.


In [ ]:
from transformers import Wav2Vec2Processor, Wav2Vec2Model
import soundfile as sf
import torch
from tqdm import tqdm
import kaldiio

# Load the processor and model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model.config.output_hidden_states = True

# Specify the target layer number (0 for embeddings, 1 for first hidden layer, etc.)
target_layer = 24  # Adjust this according to which layer you want to extract features from

# Use CUDA if available
device = torch.device("cpu")
model.to(device)

# Read the list of audio file paths with speaker IDs
file_path = 'train/wav.scp'
with open(file_path, 'r') as file:
    audio_files = file.readlines()

# Lists to store speaker IDs and features for the target layer
speaker_ids = []
layer_features = []

# Process each audio file
for audio_file in tqdm(audio_files):
    audio_file = audio_file.strip()
   
    # Extract speaker ID from the file path
    speaker_id = audio_file.split('/')[-1].split('-')[0]  # adjust based on your file path format
    speaker_ids.append(speaker_id)
   
    # Load the audio file
    audio_input, sample_rate = sf.read(audio_file)
   
    # Process the audio input
    input_values = processor(audio_input, sampling_rate=sample_rate, return_tensors="pt").input_values.to(device)
   
    # Pass through the model
    with torch.no_grad():
        outputs = model(input_values)
   
    # Get the features from the target layer
    all_hidden_states = outputs.hidden_states
    layer_feature = all_hidden_states[target_layer].squeeze(0).cpu().numpy()  # Remove batch dimension and move to CPU
    layer_features.append(layer_feature)
   
    # Release GPU memory
    del input_values, outputs
    torch.cuda.empty_cache()

# Save the features in Kaldi format
output_ark = f'layer_train_features/layer_{target_layer}_features_train.ark'
output_scp = f'layer_train_features/layer_{target_layer}_features_train.scp'

with kaldiio.WriteHelper(f'ark,scp:{output_ark},{output_scp}') as writer:
    for j, feature in enumerate(layer_features):
        writer[speaker_ids[j]] = feature

print(f"Features from layer {target_layer} saved in Kaldi format.")


Some weights of the model checkpoint at facebook/wav2vec2-large-960h-lv60-self were not used when initializing Wav2Vec2Model: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 62/62 [00:20<00:00,  3.05it/s]

Features from layer 24 saved in Kaldi format.
